In [150]:
#Data Analysis
import numpy as np
import os
import json
trainexamples = []
testexamples = []

def loadtest(direc):
    for each in os.listdir(direc):        
        if '.npy' in each:
            fids = each.split('_')[1][:-4]
            examples = np.load(direc+'/'+each)
            with open(direc+'/'+each[:-4]+'.json', 'r') as infile:
                values = json.load(infile)
            yield examples,fids,values
    return

   
def load(direc):
    examples = []
    fids  = []
    values = []
    for each in os.listdir(direc):        
        try:
            if '.npy' in each:
                fids.append(each.split('_')[1][:-4])
                examples.extend(np.load(direc+'/'+each))
                with open(direc+'/'+each[:-4]+'.json', 'r') as infile:
                    values.extend(json.load(infile))
        except:
            continue
    return examples,fids,values

trainexamples, trainfids, trainvalues = load('../../../../data/Titlefeature/cv1/train')

trainlabels, trainfeatures = [], []
for each in trainexamples:
    trainfeatures.append(each[1:])
    trainlabels.append(each[0])

testexamples, testfids, testvalues = load('../../../../data/Titlefeature/cv1/test')
testfeatures, testlabels = [], []
for each in testexamples:
    testfeatures.append(each[1:])
    testlabels.append(each[0])
assert(len(trainvalues)==len(trainlabels))
assert(len(testvalues)==len(testlabels))
print len(trainfeatures[0])
print len(trainfeatures)

2
7485


In [151]:
print trainlabels.count(1), trainlabels.count(2)

29 420


In [2]:
#Content extractor
#non parametric classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
import numpy as np

testlabels_title = map(lambda x: 1 if x==2 else 0 , testlabels)
p_dectree =  np.array(testfeatures)[:,0]
print precision_score(p_dectree,testlabels_title)
cnt = 0
print '---------Title---------'
for i,each in enumerate(zip(p_dectree,testlabels_title)):
    if each[0] == 0 and each[1]==1:
        print 'Mine %d Ground %d'%(each[0],each[1])
        print '-------------Error number %d------------' %cnt
        print testvalues[i]
        print testfeatures[i]
        cnt+=1



NameError: name 'testlabels' is not defined

In [1]:
class TitleClassifier:
    
    def __init__(self):
        """Predicts for a single news article only"""
        pass
    
    def predict(self,X):        
        assert(X.shape[1] == 2)        
        rowno = 0
        res = -1
        gotit = False
        for goodmatch,isheading in X:
            
            if goodmatch == 1 and isheading==1:
                res = rowno
                
                break;
            if isheading == 1 and not gotit:
                res = rowno
                gotit  = True
            rowno+=1
        Y = [0]*X.shape[0]
        print res
        if res>=0:
            Y[res] = 1
        return Y
            
TestCheck = TitleClassifier()
import pickle
pickle.dump( TestCheck, file('Best_title.dat','wb'))
TC = pickle.load(file('Best_title.dat','rb'))
for x,y,v in loadtest('../../../../data/Titlefeature/cv1/test'):
    Y = TC.predict(np.array(x).transpose()[1:,].transpose())
    print '--**'*10
    title = ''
    for i,e in enumerate(Y):
        if e == 1:
            title+=v[i]
    print title
        
                
                
        

NameError: name 'loadtest' is not defined